In [1]:
from sentence_transformers import SentenceTransformer
from collections import Counter
import requests
import json
from tqdm import tqdm
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Change to the parent directory of the notebook

import os
os.chdir('..')
os.chdir('..')


TRAIN_DF = pd.read_csv("Data/TRAIN_DF.csv", index_col=0)


In [5]:
TRAIN_DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9000 entries, 0 to 8999
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   post_id                               9000 non-null   object 
 1   tweet_text                            9000 non-null   object 
 2   key_features                          9000 non-null   object 
 3   target                                6059 non-null   object 
 4   label                                 9000 non-null   object 
 5   Hermes-3-Llama-3.1-70B-Q5_K_S         9000 non-null   object 
 6   llama3.3:70B-Instruct-Q2_K            9000 non-null   object 
 7   llama3.3:70b-instruct-q6_K-SEEN_DATA  9000 non-null   object 
 8   llama3.3:70b-instruct-q6_K            9000 non-null   object 
 9   mistral:7b-instruct-v0.2-q8_0         9000 non-null   object 
 10  X_train                               9000 non-null   object 
 11  Row Number            

In [7]:
TRAIN_DF.head()

,post_id,tweet_text,key_features,target,label,Hermes-3-Llama-3.1-70B-Q5_K_S,llama3.3:70B-Instruct-Q2_K,llama3.3:70b-instruct-q6_K-SEEN_DATA,llama3.3:70b-instruct-q6_K,mistral:7b-instruct-v0.2-q8_0,X_train,Row Number,Input,Explanations,Time Taken (s),label_encoded
0,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,[],NaN,normal,offensive,normal,normal,normal,normal,"""""""tweet_text : i dont think im getting my bab...",1,"{'post_id': '1179055004553900032_twitter', 'tw...",The text appears to be a personal statement ab...,39.40,1
1,1179063826874032128_twitter,we cannot continue calling ourselves feminists...,[],NaN,normal,normal,normal,normal,normal,offensive,"""""""tweet_text : we cannot continue calling our...",2,"{'post_id': '1179063826874032128_twitter', 'tw...",The text falls under the category of normal. H...,18.88,1
2,1178793830532956161_twitter,nawt yall niggers ignoring me,[],"African, None",normal,hatespeech,hatespeech,hatespeech,hatespeech,offensive,"""""""tweet_text : nawt yall niggers ignoring me\...",3,"{'post_id': '1178793830532956161_twitter', 'tw...",This text cannot be classified as normal. The ...,13.28,1
3,1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,"['ching', 'chong']",Asian,hatespeech,offensive,hatespeech,hatespeech,hatespeech,offensive,"""""""tweet_text : <user> i am bit confused coz c...",4,"{'post_id': '1179088797964763136_twitter', 'tw...",The text is classified as hate speech due to t...,27.10,0
4,1179085312976445440_twitter,this bitch in whataburger eating a burger with...,"['bitch', 'i', 'hate', 'white', 'bitches']","Caucasian, Women",hatespeech,hatespeech,hatespeech,hatespeech,hatespeech,offensive,"""""""tweet_text : this bitch in whataburger eati...",5,"{'post_id': '1179085312976445440_twitter', 'tw...",**Hate Speech**:\n\nThe tweet explicitly emplo...,18.15,0


In [6]:
TRAIN_DF['X_train'] = (
    '"""tweet_text : ' + TRAIN_DF['tweet_text'] + '\n' +
    'target: ' + TRAIN_DF['target'].fillna('') + '\n' +
    'explanations: ' + TRAIN_DF['Explanations'] + '"""'
)

In [8]:
EMBEDDINGS_MODEL = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True, device="cuda").cuda()

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_1.5B_v5:
- modeling_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_1.5B_v5:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [9]:
# print("generating embeddings...")
# Generate embeddings for each row in the 'tweet_text' column
tqdm.pandas()

# Generate embeddings with progress tracking
TRAIN_DF['X_train_emebeddings'] = TRAIN_DF['X_train'].progress_apply(lambda text: EMBEDDINGS_MODEL.encode(text))

# df.to_pickle("dataset_mit_embeddings.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9000/9000 [08:35<00:00, 17.44it/s]


In [11]:
TEST_DF = pd.read_csv("Data/TEST_DF.csv", index_col=0)

In [17]:
TEST_DF['X_train_emebeddings'] = TEST_DF['tweet_text'].progress_apply(lambda text: EMBEDDINGS_MODEL.encode(text))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1884/1884 [01:29<00:00, 21.02it/s]


In [10]:
import xgboost as xgb

In [18]:
label_encoder = LabelEncoder()
TRAIN_DF['label_encoded'] = label_encoder.fit_transform(TRAIN_DF['label'])
TEST_DF['label_encoded'] = label_encoder.transform(TEST_DF['label'])

# Step 3: Convert 'X_train' column to a list of embeddings
X_train = TRAIN_DF['X_train_emebeddings'].tolist()
X_test = TEST_DF['X_train_emebeddings'].tolist()
y_train = TRAIN_DF['label_encoded'].tolist()
y_test = TEST_DF['label_encoded'].tolist()



# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Use 'multi:softmax' for multi-class classification
    num_class=len(label_encoder.classes_),  # Number of classes
    eval_metric='mlogloss',  # Multi-class log loss
    use_label_encoder=False  # Avoid warnings about label encoding
)

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:00:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=3, num_parallel_tree=None, ...)

In [19]:
TEST_DF['X_train']

0       [ 6.2125096  -0.68465716 -6.3372498  ... 10.51...
1       [ 0.7899033  1.2117541 -4.710365  ...  6.32203...
2       [ 0.08608349  0.26262066 -6.3408113  ... 10.52...
3       [-0.6398999   0.87283546  0.4512122  ... 11.27...
4       [ 1.7874744  -0.0422212  -3.9652562  ...  6.74...
                              ...                        
1879    [ 1.3682098  4.4005337  2.4642673 ...  8.22857...
1880    [-0.6184644   0.36710113 -1.5807176  ...  5.76...
1881    [ 3.8049612   2.09012     1.3787781  ...  6.64...
1882    [ 1.3526949   0.62941974 -6.9584336  ... 11.17...
1883    [ 0.2444589  2.437378   1.1887732 ... 10.19462...
Name: X_train, Length: 1884, dtype: object

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.3689

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.51      0.20      0.29       628
      normal       0.35      0.88      0.50       628
   offensive       0.28      0.02      0.04       628

    accuracy                           0.37      1884
   macro avg       0.38      0.37      0.28      1884
weighted avg       0.38      0.37      0.28      1884

